In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import SimpleITK as sitk
import re
import numpy as np
import matplotlib.pyplot as plt
import sitk_helper as sitkh

In [ ]:
raw_images = <pathToTRUFIData>
output = 'External TRUFI/second_set/Cleaned/'
testImagePath = 'External TRUFI/second_set/imagesTs/'
testLabelPath = 'External TRUFI/second_set/labelsTs/'
testPredictPath = 'External TRUFI/second_set/labelsTsPredict/'

## Locate the different raw source files 

In [ ]:
from pathlib import Path

In [ ]:
if not os.path.exists(output):
        os.mkdir(output)
rawdata = []

path = Path(raw_images)
files = [file for file in [str(pp) for pp in path.glob("*/*.nii")] if os.path.isfile(file)]

ids = np.unique([int(re.search(r'\d{3}', file).group(0)) for file in files])
for num in ids:
    id_files = [file for file in files if str(num) in file]
    #print(id_files)
    print(num)

    cartilage = [file for file in id_files if 'art' in file][0]
    labrum = [file for file in id_files if 'abr' in file][0]
    #bones = [file for file in files if 'Bones' in file][0]
    image = [file for file in id_files if file != cartilage and file != labrum][0]

    outputfolder = os.path.join(output,'{0:03d}'.format(num))

    if not os.path.exists(outputfolder):
        os.mkdir(outputfolder)
    rawdata.append({'ID':num,
                 'rawImage':os.path.join(raw_images,image),
                 'rawCartilage':os.path.join(raw_images,cartilage),
                 'rawLabrum':os.path.join(raw_images,labrum),
                 'image':os.path.join(outputfolder,'{0:03d}_T2.nii.gz'.format(num)),
                 'labels':os.path.join(outputfolder,'{0:03d}_Labels.nii.gz'.format(num))
        })
rawdata = sorted(rawdata, key=lambda d: d['ID'])
for i, patient in enumerate(rawdata):
    patient['index'] = i
    print(patient['ID'], patient['index'] )

rawdata

## Merge labels and unify index (plane) order

In [ ]:
def sphereFit(spX,spY,spZ):
    #   Assemble the A matrix
    spX = np.array(spX)
    spY = np.array(spY)
    spZ = np.array(spZ)
    A = np.zeros((len(spX),4))
    A[:,0] = spX*2
    A[:,1] = spY*2
    A[:,2] = spZ*2
    A[:,3] = 1

    #   Assemble the f matrix
    f = np.zeros((len(spX),1))
    f[:,0] = (spX*spX) + (spY*spY) + (spZ*spZ)
    C, residules, rank, singval = np.linalg.lstsq(A,f)

    #   solve for the radius
    t = (C[0]*C[0])+(C[1]*C[1])+(C[2]*C[2])+C[3]
    radius = np.sqrt(t)

    return C[0:3].T[0], radius

In [ ]:
sitk.ProcessObject.SetGlobalDefaultCoordinateTolerance(1e-3)
for patient in rawdata:
    
    cartilage = sitk.ReadImage(patient['rawCartilage'])
    labrum = sitk.ReadImage(patient['rawLabrum'])
    
    phy_center = np.array(cartilage.TransformContinuousIndexToPhysicalPoint((np.array(cartilage.GetSize())/2).tolist()))
    
    cartilage_np = sitk.GetArrayFromImage(cartilage)
    labrum_np = sitk.GetArrayFromImage(labrum)


    label_np = np.zeros_like(cartilage_np) #1: cartilage, 2: labrum
    label_np[cartilage_np == 1] = 1
    label_np[labrum_np == 1] = 2
    
    #label = cartilage + 2 * labrum
    label = sitk.GetImageFromArray(label_np)
    label.CopyInformation(cartilage)
    image = sitk.ReadImage(patient['rawImage'])
        
    image = sitkh.AmiraFlip(image)
    label = sitkh.AmiraFlip(label)

    reoriented_scan = sitk.DICOMOrient(image,"LPS")
    reoriented_label = sitk.DICOMOrient(label,"LPS")

    if reoriented_scan.TransformContinuousIndexToPhysicalPoint(np.asarray(reoriented_scan.GetSize()) /2)[0] < 0:
        print(patient["ID"]," Right")
        patient["side"] = "Right"
        if reoriented_scan.GetSpacing()[0] < reoriented_scan.GetSpacing()[2]:
            print("lower than 45°", patient["ID"])
            reoriented_scan = sitk.DICOMOrient(image,"IPL")
            reoriented_label = sitk.DICOMOrient(label,"IPL")
    else:
        print(patient["ID"]," Left")
        patient["side"] = "Left"
        if reoriented_scan.GetSpacing()[0] < reoriented_scan.GetSpacing()[2]:
            print("lower than 45°", patient["ID"])
            reoriented_scan = sitk.DICOMOrient(image,"SPR")
            reoriented_label = sitk.DICOMOrient(label,"SPR")
            #reoriented_scan = sitk.DICOMOrient(image,"IPR")
            #reoriented_label = sitk.DICOMOrient(label,"IPR")
            # RPS and IPR comments would make that the left and right looks the same for the network
            
    input_image = sitk.GetArrayFromImage(reoriented_scan)
    input_label = sitk.GetArrayFromImage(reoriented_label)
    
    sitk.WriteImage(reoriented_scan,patient["image"])
    sitk.WriteImage(reoriented_label,patient["labels"]) 

In [ ]:
coordinates = np.zeros([len(rawdata),4])
for patient in rawdata:    
    segmentation = sitk.ReadImage(patient['labels'])

    segmentation_np = sitk.GetArrayFromImage(segmentation)

    PhysicalPointsSource = sitk.PhysicalPointImageSource()
    PhysicalPointsSource.SetReferenceImage(segmentation)
    PhysicalPoints = PhysicalPointsSource.Execute()
    segmentation_physical_np = sitk.GetArrayFromImage(PhysicalPoints)
    verticesCartilage = segmentation_physical_np[np.where(segmentation_np == 1)]

    center,radius = sphereFit(verticesCartilage[:,0],verticesCartilage[:,1],verticesCartilage[:,2])
    coordinates[patient['index'],0] = patient['ID']
    coordinates[patient['index'],1:4] = center
    
print(coordinates)
np.savetxt(os.path.join(output,'femur_head_center_phy.txt'),coordinates , fmt=('%.0f','%.2f','%.2f','%.2f'),delimiter=', ')  

In [ ]:
print(not_aligned)
not_aligned = []

In [ ]:
sitk.ProcessObject.SetGlobalDefaultCoordinateTolerance(1e-3)
head_centers = np.loadtxt(os.path.join(output,'femur_head_center_phy.txt'),delimiter = ',',dtype='float')

rawdata_perside = sorted(rawdata, key=lambda d: d['side'])
for patient in rawdata_perside:
    if patient['ID'] in not_aligned:
        continue
    #visualize planes
    print(patient['ID'], patient['side'])
    reoriented_scan = sitk.ReadImage(patient['image'])
    input_image = sitk.GetArrayFromImage(reoriented_scan)
    input_label = sitk.GetArrayFromImage(sitk.ReadImage(patient['labels']))
    
    fig = plt.figure(figsize=(15, 5))
    index = np.where(head_centers[:,0] == patient['ID'])
    center_phy = head_centers[index[0],1:4][0]
    center = reoriented_scan.TransformPhysicalPointToIndex(center_phy.tolist())
    
    
    for i in range(3):
        ax = fig.add_subplot(1, 3, i+1, xticks=[], yticks=[]) 

        
        img = np.take(input_image, center[i],2-i)
        lab = np.take(input_label, center[i],2-i)
        
        alphas = np.ones(lab.shape)
        alphas[lab==0] = 0
        plt.imshow(img,cmap='gray', vmin=0, vmax=200,aspect = 'auto')
        plt.imshow(lab, cmap='tab10',alpha =alphas,aspect = 'auto')
        ax.axis('off')

        fig.tight_layout()
        fig.subplots_adjust(wspace=0, hspace=0)
        ax.set_title("{}: Center {}, {}, {}".format(
            patient["ID"],center_phy[0],center_phy[1],center_phy[2]),color="black")
    plt.show()   

## Locate the different unified files 

In [ ]:
data = []
dirs = [dir_ for dir_ in  os.listdir(output) if os.path.isdir(os.path.join(output,dir_))]
for dir_ in dirs:
    files = os.listdir(os.path.join(output,dir_))
    image = [file for file in files if 'T2' in file][0]
    label = [file for file in files if 'Labels' in file][0]

    data.append({'ID':int(re.search(r'\d{3}', image).group(0)),
                 'sourceImagePath':os.path.join(output,dir_,image),
                 'sourceLabelPath':os.path.join(output,dir_,label)
        })
data = sorted(data, key=lambda d: d['ID'])
for i, patient in enumerate(data):
    patient['index'] = i
    print(patient['ID'], patient['index'] )
data

In [ ]:
train_id = []
test_id = ids

### Define path and names for nnUnet 

In [ ]:
def checkAndCreateFolder(path):
    if not os.path.exists(path):
        os.makedirs(path)
            
checkAndCreateFolder(testImagePath)
checkAndCreateFolder(testLabelPath)

for patient in data:
    imageName = 'labrum_{:03d}_0000.nii.gz'.format(patient['ID'])
    labelName = 'labrum_{:03d}.nii.gz'.format(patient['ID'])
    if patient['ID'] in train_id:
        patient['destImagePath'] = os.path.join(trainImagePath,imageName)
        patient['destLabelPath'] = os.path.join(trainLabelPath,labelName)
        patient['Set'] = 'Train'
    if patient['ID'] in test_id:
        patient['destImagePath'] = os.path.join(testImagePath,imageName)
        patient['destLabelPath'] = os.path.join(testLabelPath,labelName)
        patient['Set'] = 'Test'

In [ ]:
def checkPhysicalspace(image1,image2):
    origin = np.linalg.norm(np.asarray(image1.GetOrigin())-np.asarray(image2.GetOrigin()))
    spacing = max(abs(np.asarray(image1.GetSpacing())-np.asarray(image2.GetSpacing())))
    direction = max(abs(np.asarray(image1.GetDirection())-np.asarray(image2.GetDirection())))
    assert origin < 1e-3, "Origin difference {:e}".format(origin)
    assert spacing < 1e-3, "Spacing difference {:e}".format(origin)
    assert direction < 1e-3, "Direction difference {:e}".format(origin)


### Crop and export data

In [ ]:
head_centers = np.loadtxt(os.path.join(output,'femur_head_center_phy.txt'),delimiter = ',',dtype='float')

for patient in data:
    
    image = sitk.ReadImage(patient['sourceImagePath'])
    labels = sitk.ReadImage(patient['sourceLabelPath'])
    
    index = np.where(head_centers[:,0] == patient['ID'])
    center_phy = head_centers[index[0],1:4][0]
    
    dest_spacing = np.array([1.0, 0.5, 0.5])
    dev = np.abs(np.array(image.GetSpacing()) - dest_spacing)/dest_spacing
    print(dev)
    if np.max(dev) > 0.15:
        print('resampled')
        labels = sitkh.changeImageOrientationAndSpacing(labels, new_spacing = dest_spacing, interpolator = sitk.sitkNearestNeighbor)
        image = sitkh.changeImageOrientationAndSpacing(image, new_spacing = dest_spacing)
    
    crop_i  = sitkh.cropRoi(image,phyCenter=center_phy, destSize = [80,200,200])
    crop_l = sitkh.cropRoi(labels,phyCenter=center_phy, destSize = [80,200,200] ,checkLabels =True)
    
    checkPhysicalspace(crop_i,crop_l)
    crop_l.SetSpacing(crop_i.GetSpacing())
    crop_l.SetOrigin(crop_i.GetOrigin())
    crop_l.SetDirection(crop_i.GetDirection())
    
    crop_image = sitk.GetArrayFromImage(crop_i)
    crop_label = sitk.GetArrayFromImage(crop_l)
    
    found_labels = np.unique(crop_label)
    if not np.isin(found_labels, np.arange(0,5)).all() or np.isin(found_labels, np.arange(5,10)).any():
        print("{} -> found labels {}".format(patient['ID'],found_labels))
        #crop_label[crop_label == 3] = 2
        crop_l = sitk.GetImageFromArray(crop_label)
        crop_l.SetSpacing(crop_i.GetSpacing())
        crop_l.SetOrigin(crop_i.GetOrigin())
        crop_l.SetDirection(crop_i.GetDirection())
    
    sitk.WriteImage(crop_i,patient['destImagePath'])
    sitk.WriteImage(crop_l,patient['destLabelPath'])
    
    fig = plt.figure(figsize=(15, 5))
    
    for i in range(3):        
        ax = fig.add_subplot(1, 3, i+1, xticks=[], yticks=[]) 
        
        
        img = np.take(crop_image, crop_image.shape[2-i]/2,2-i)
        lab = np.take(crop_label, crop_label.shape[2-i]/2,2-i)
        
        alphas = np.ones(lab.shape)
        alphas[lab==0] = 0
        plt.imshow(img,cmap='gray', vmin=0, vmax=200,aspect = 'auto')
        plt.imshow(lab, cmap='tab10',alpha =alphas,aspect = 'auto')
        ax.axis('off')
        
        
        
        fig.tight_layout()
        fig.subplots_adjust(wspace=0, hspace=0)
        ax.set_title("{}: Center {}, {}, {}".format(
            patient["ID"],center_phy[0],center_phy[1],center_phy[2]),color="black")
    plt.show()

added to retrain

In [ ]:
import os
from collections import OrderedDict
import json
output_folder = "/mnt/Data/nnUNet_raw/nnUNet_raw_data/Task604_Cartilage_Labrum_TRUFI/"
train_ids = []
test_ids = []
calib_ids = []
for fileName in os.listdir(output_folder+'/imagesTr'):
    train_ids.append(fileName.split('_0000')[0])

if os.path.exists(output_folder+'/imagesTs'):
    for fileName in os.listdir(output_folder+'/imagesTs'):
        test_ids.append(fileName.split('_0000')[0])
        
if os.path.exists(output_folder+'/imagesCalib'):
    for fileName in os.listdir(output_folder+'/imagesCalib'):
        calib_ids.append(fileName.split('_0000')[0])

json_dict = OrderedDict()
json_dict['name'] = "Cartilage_Labrum_Bone"
json_dict['description'] = "Cropped TRUFI segmenation with cartilage, labrum. To test how much we can improve when retained on TRUFI data (20)"
json_dict['tensorImageSize'] = "3D"
json_dict['reference'] = "-"
json_dict['licence'] = "-"
json_dict['release'] = "-"
json_dict['modality'] = {
    "0": "MRI"
}

json_dict['labels'] = {
    "0": "background",
    "1": "cartilage",
    "2": "labrum",
}

json_dict['numTraining'] = len(train_ids)
json_dict['numTest'] = len(test_ids)
json_dict['training'] = [{'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i in train_ids]
json_dict['test'] = ["./imagesTs/%s.nii.gz" % i for i in test_ids]
json_dict['calibration'] = ["./imagesCalib/%s.nii.gz" % i for i in calib_ids]

with open(os.path.join(output_folder, "dataset.json"), 'w') as f:
    json.dump(json_dict, f, indent=4, sort_keys=True)